# Pre-Process Data: Specific DSAID

In [2]:
"""Pre-Process Data

Convert the raw data counts into sc-RNAseq compatible data format.

Each dataset may have many different samples. We may decide to split the data taking this into consideration or not. 

We will here contemplate 3 types of train/test/validation splits:
    1. 0 Dataset Leakage: No overlap between datasets
    2. Train-Test Dataset Leakage: Overlap between train and test datasets
    3. Train-Test-Validation Dataset Leakage: Overlap between train, test, and validation datasets


Structure:
    1. Imports, Variables, Functions
    2. Load Data
    3. Convert to `adata` object

"""

# 1. Imports, Variables, Functions
# imports
import numpy as np, os, sys, pandas as pd, scanpy as sc
import anndata as ad

# variables
example_data_path = (
    "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/tmp/DSA00015.csv"
)

# functions


# 2. Load Data
df = pd.read_csv(example_data_path, index_col=0)

# 3. Convert to `adata` object


# Extract cell identifiers and gene expression data
cell_ids = df.iloc[:, 0]
gene_expression_data = df.iloc[:, 1:].values
gene_names = df.columns[1:]


# Create an AnnData object
adata = ad.AnnData(X=gene_expression_data)


# Add cell and gene metadata
adata.obs["cell_ids"] = cell_ids.values
adata.var["gene_symbols"] = gene_names

# Preprocess Data: Specific Diseases

In [3]:
"""Pre-Process Data

Convert the raw data counts into sc-RNAseq compatible data format.

Structure:
    1. Imports, Variables, Functions
    2. Load Data
    3. Convert to `adata` object

"""

# 1. Imports, Variables, Functions
# imports
import numpy as np, os, sys, pandas as pd, scanpy as sc
import anndata as ad
import logging
from tqdm import tqdm

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
from matplotlib import pyplot as plt
from collections import Counter

# import random
import random

from sklearn.model_selection import train_test_split


# variables
diseases_of_interest_set = {"Influenza", "Colorectal Carcinoma", "Breast Cancer"}
diseases_of_interest_set = {
    "Crohn's Disease",
    "Ulcerative Colitis",
    "Lung Cancer",
    "Lung Adenocarcinoma",
    "Breast Cancer",
    "Psoriasis",
}


example_data_path = (
    "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/tmp/DSA00123.csv"
)

df_info_path = os.path.join(
    "/aloy",
    "home",
    "ddalton",
    "projects",
    "disease_signatures",
    "data",
    "DiSignAtlas",
    "Disease_information_Datasets_extended.csv",
)


large_df_path = "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/DiSignAtlas.exp_prof_merged.csv"


# functions
def get_exp_prof(dsaids_interest):
    """Get Expression Profiles"""

    # variables
    file_dir = "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/tmp/"
    first = True
    for dsaid in tqdm(dsaids_interest):
        __df = pd.read_csv(os.path.join(file_dir, f"{dsaid}.csv"))
        if first:
            df_global = __df
            first = False
        else:
            df_global = pd.concat([df_global, __df], axis=0)
    return df_global


def get_dataset_info(ids, df_info):
    """Get Disease Information"""
    dsaid_2_accession = dict(zip(df_info["dsaid"], df_info["accession"]))

    dataset_accessions = [dsaid_2_accession[id.split(";")[0]] for id in ids]

    accession_2_id = {k: v for v, k in enumerate(set(dataset_accessions))}
    dataset_ids = [accession_2_id[accession] for accession in dataset_accessions]

    return dataset_accessions, dataset_ids


def get_disease_info(ids, df_info):
    """Get Disease Information"""
    dsaid_2_disease = dict(zip(df_info["dsaid"], df_info["disease"]))
    disease_types = list()
    for id in ids:
        if id.split(";")[2] == "Control":
            disease_types.append("Control")
        elif id.split(";")[2] == "Case":
            disease_types.append(dsaid_2_disease[id.split(";")[0]])
        else:
            print(f"Error with ID: {id}")

    map_2_id = {k: v for v, k in enumerate(set(disease_types))}

    disease_ids = [map_2_id[x] for x in disease_types]

    return disease_types, disease_ids


# 2. Load Data
# df = pd.read_csv(example_data_path)

df_info = pd.read_csv(df_info_path)


# Query data to retrieve dsaids of interest
library_strategies_of_interest_set = {"RNA-seq", "Microarray"}
QUERY = "disease in @diseases_of_interest_set & library_strategy in @library_strategies_of_interest_set & organism == 'Homo sapiens'"
dsaids_interest = df_info.query(QUERY)["dsaid"].to_list()
logging.info(f"Nº of DSAIDs of interest: {len(dsaids_interest)}")

# skip_rows_idxs = get_skip_rows(dsaids_interest)

df = get_exp_prof(dsaids_interest)

# 3. Convert to `adata` object
# load dataframe
# df = pd.read_csv(large_df_path, skiprows=skip_rows_idxs, index_col=0)
logging.info(f"Loaded dataframe with shape: {df.shape}")

# drop non significant rows
# Calculate the number of NaNs in each row
nan_counts = df.isna().sum(axis=1)

# Filter the DataFrame to keep only rows with NaNs less than or equal to 18,000
df = df[nan_counts <= 18000]
logging.info(f"Filtered dataframe with shape: {df.shape}")

# Filter out Unknown samples
mask = [False if id.split(";")[2] == "Unknown" else True for id in df.iloc[:, 0].values]

df = df[mask]

logging.info(f"Filtered out Unkowns from dataframe with shape: {df.shape}")


# 3. Convert to `adata` object
# Extract cell identifiers and gene expression data
ids = df.iloc[:, 0]
gene_expression_data = df.iloc[:, 1:].values
gene_names = df.columns[1:]

2024-08-14 16:13:51,285 - Nº of DSAIDs of interest: 286
100%|██████████| 286/286 [03:13<00:00,  1.48it/s]
2024-08-14 16:17:04,621 - Loaded dataframe with shape: (10963, 19691)
2024-08-14 16:17:05,040 - Filtered dataframe with shape: (10791, 19691)
2024-08-14 16:17:05,380 - Filtered out Unkowns from dataframe with shape: (10583, 19691)


In [25]:
# Create an AnnData object
adata = ad.AnnData(X=gene_expression_data)


# Add cell and gene metadata
adata.obs["ids"] = ids.values
adata.var["gene_symbols"] = gene_names
adata.var["index"] = gene_names
adata.var["gene_name"] = gene_names

In [26]:
# generate celltype and celltype_id columns in adata.obs
disease_types, disease_ids = get_disease_info(ids.values, df_info=df_info)
adata.obs["celltype"] = disease_types
adata.obs["celltype_id"] = disease_ids

logging.info(f"Counts of each disease type: {Counter(disease_types)}")

2024-08-14 16:33:21,681 - Counts of each disease type: Counter({'Control': 4041, 'Ulcerative Colitis': 1776, 'Breast Cancer': 1740, "Crohn's Disease": 1469, 'Psoriasis': 960, 'Lung Adenocarcinoma': 335, 'Lung Cancer': 262})


In [27]:
# generate dataset ids column in adata.obs
dataset_ids, batch_ids = get_dataset_info(ids.values, df_info=df_info)

adata.obs["dataset_id"] = dataset_ids
adata.obs["batch_id"] = batch_ids

logging.info(f"Counts of each dataset id: {Counter(dataset_ids)}")

2024-08-14 16:33:23,120 - Counts of each dataset id: Counter({'GSE206285': 566, 'GSE13355': 468, 'GSE66407': 452, 'GSE54002': 431, 'GSE112366': 410, 'GSE119600': 278, 'GSE87650': 267, 'GSE48634': 235, 'GSE75214': 224, 'GSE179285': 201, 'GSE5364': 194, 'GSE40791': 192, 'GSE97012': 191, 'GSE123086': 188, 'GSE4115': 185, 'GSE92415': 181, 'GSE20881': 168, 'GSE3365': 165, 'GSE14905': 163, 'GSE45827': 162, 'GSE136757': 159, 'GSE20189': 155, 'GSE13367': 149, 'GSE27562': 146, 'GSE87473': 144, 'GSE24124': 135, 'GSE126124': 123, 'GSE24287': 120, 'GSE16443': 119, 'GSE42568': 119, 'GSE29044': 107, 'GSE37751': 106, 'GSE83582': 106, 'GSE10072': 105, 'GSE60083': 98, 'GSE94648': 96, 'GSE192867': 90, 'GSE18842': 89, 'GSE7670': 84, 'GSE6710': 77, 'GSE55201': 72, 'GSE38713': 69, 'GSE43458': 68, 'GSE95095': 68, 'GSE29431': 64, 'GSE32407': 60, 'GSE61304': 58, 'GSE42826': 58, 'GSE102133': 56, 'GSE10810': 56, 'GSE71730': 56, 'GSE75343': 54, 'GSE105074': 52, 'GSE53306': 52, 'GSE83448': 51, 'GSE110933': 48, 'G

# Perform Splits

In [32]:
# copy
df_val_split = adata.obs.copy()

(10583, 5)
(164, 2)


In [65]:
df_test

,celltype,dataset_id
29,Lung Adenocarcinoma,GSE7670
99,Breast Cancer,GSE9574
126,Psoriasis,GSE6710
167,Psoriasis,GSE136757
301,Psoriasis,GSE185764
...,...,...
10240,Psoriasis,GSE160932
10258,Breast Cancer,GSE45827
10415,Lung Adenocarcinoma,GSE10072
10456,Breast Cancer,GSE25487


In [89]:
from typing import *
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")


def test_split(
    df_obs: pd.DataFrame, ratio: float = 0.15, random_state: int = 42
) -> List[bool]:
    """
    Splits the dataset into a validation set based on the specified ratio.

    Args:
        df_obs (pd.DataFrame): DataFrame containing the observational data.
        ratio (float): The ratio of the dataset to be used for validation.

    Returns:
        mask (List[bool]): A boolean mask indicating which samples belong to the validation set.
    """

    df_val_split = df_obs.copy()

    # Columns of interest
    interest_columns = ["celltype", "dataset_id"]

    # Remove unnecessary columns
    columns_to_remove = list(set(df_val_split.columns) - set(interest_columns))
    df_val_split.drop(columns=columns_to_remove, inplace=True)

    # Filter out control samples and remove duplicates
    df_val_split = df_val_split[df_val_split["celltype"] != "Control"].drop_duplicates()

    # Split the data into training and validation sets
    X = df_val_split["dataset_id"].values
    y = df_val_split["celltype"].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=ratio, random_state=random_state
    )

    # Check for dataset leakage
    dataset_leakage = set(X_train) & set(X_test)
    logging.info(f"Data leakage: {len(dataset_leakage)}")

    logging.info(f"Train: {len(X_train)}, Test: {len(X_test)}")

    # Remove leakage from training set
    X_train = list(set(X_train) - dataset_leakage)

    dataset_leakage = set(X_train) & set(X_test)
    assert len(dataset_leakage) == 0, "Dataset leakage still present"

    # Check for sufficient data in each split
    train_diseases = df_val_split[df_val_split["dataset_id"].isin(X_train)]["celltype"]
    test_diseases = df_val_split[df_val_split["dataset_id"].isin(X_test)]["celltype"]

    assert len(set(train_diseases)) == len(
        set(test_diseases)
    ), "Train and test diseases differ"

    logging.info(
        f"Number of diseases in train: {len(set(train_diseases))}, "
        f"Number of diseases in test: {len(set(test_diseases))}"
    )
    logging.info(f"Train disease-dataset counts: {Counter(train_diseases)}")
    logging.info(f"Test disease-dataset counts: {Counter(test_diseases)}")

    # Generate the validation set mask
    mask = df_obs["dataset_id"].isin(X_test)

    logging.info(
        f"Original train/test ratio: {ratio}\n"
        f"Dataset train/test ratio: {len(X_test) / (len(X_train)+len(X_test)):.2f}\n"
        f"Sample Ratio: {mask.sum() / len(mask):.2f}"
    )

    return mask


mask_validation = test_split(df_obs=adata.obs, ratio=0.3, random_state=None)

2024-08-14 18:34:01,993 - Data leakage: 5
2024-08-14 18:34:01,993 - Train: 114, Test: 50
2024-08-14 18:34:01,994 - Number of diseases in train: 6, Number of diseases in test: 6
2024-08-14 18:34:01,994 - Train disease-dataset counts: Counter({'Breast Cancer': 29, 'Psoriasis': 23, 'Ulcerative Colitis': 21, "Crohn's Disease": 19, 'Lung Adenocarcinoma': 9, 'Lung Cancer': 8})
2024-08-14 18:34:01,995 - Test disease-dataset counts: Counter({'Ulcerative Colitis': 14, 'Psoriasis': 12, 'Breast Cancer': 12, "Crohn's Disease": 11, 'Lung Adenocarcinoma': 5, 'Lung Cancer': 1})
2024-08-14 18:34:01,995 - Original train/test ratio: 0.3
Dataset train/test ratio: 0.34
Sample Ratio: 0.30


In [5]:
# We combine both the disease type with the dataset as to
# shuffle even more the data - minimize bias in train/test split
labels = np.array([a + b for a, b in zip(disease_types, dataset_ids)])

# Generate indices for the data points
indices = np.arange(len(labels))

"""train_test_split cannot handle single label indexes

Because of this we will manually deal with these cases!

"""
indices_single_label = [i for i, x in enumerate(labels) if list(labels).count(x) == 1]
labels_single_label = labels[indices_single_label]

logging.info(f"Nº of single label indexes: {len(indices_single_label)}")


remaining_indices = [i for i in indices if i not in indices_single_label]
remaining_labels = labels[remaining_indices]

# Perform stratified split on the remaining indices
train_indices, test_indices = train_test_split(
    remaining_indices, test_size=0.2, stratify=remaining_labels, random_state=42
)

for idx in indices_single_label:

    if random.random() < 0.2:
        test_indices.append(idx)
    else:
        train_indices.append(idx)


test_indices.sort()
train_indices.sort()

batch_ids = np.zeros(len(labels))
batch_ids[test_indices] = 1


logging.info(f"Nº of train samples: {np.sum(~batch_ids.astype(bool))}")
logging.info(f"Nº of test samples: {np.sum(batch_ids.astype(bool))}")


adata.obs["train_test"] = batch_ids
adata.obs["str_batch"] = batch_ids.astype(int).astype(str)

2024-08-14 14:40:37,048 - Nº of single label indexes: 1
2024-08-14 14:40:37,057 - Nº of train samples: 8466
2024-08-14 14:40:37,058 - Nº of test samples: 2117


In [ ]:
# save adata object
# adata.write("../data/test_1/test_1.h5ad")
adata.write("../data/test_1/test_2.h5ad")

... storing 'str_batch' as categorical


In [ ]:
adata.obs

,ids,celltype,celltype_id,dataset_id,batch_id,train_test,batch_str,str_batch
0,DSA00057;GSM2062351;Control,Control,0,GSE77953,23,0.0,0.0,0.0
1,DSA00057;GSM2062352;Control,Control,0,GSE77953,23,1.0,1.0,1.0
2,DSA00057;GSM2062353;Control,Control,0,GSE77953,23,0.0,0.0,0.0
3,DSA00057;GSM2062354;Control,Control,0,GSE77953,23,0.0,0.0,0.0
4,DSA00057;GSM2062355;Control,Control,0,GSE77953,23,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
5085,DSA10295;GSM1012438;Case,Colorectal Carcinoma,3,GSE41258,16,0.0,0.0,0.0
5086,DSA10295;GSM1012444;Case,Colorectal Carcinoma,3,GSE41258,16,1.0,1.0,1.0
5087,DSA10295;GSM1012448;Case,Colorectal Carcinoma,3,GSE41258,16,0.0,0.0,0.0
5088,DSA10295;GSM1012449;Case,Colorectal Carcinoma,3,GSE41258,16,0.0,0.0,0.0


In [ ]:
# we will create an "artifical" cell type which
# will represent the diseases